In [ ]:
from dotenv import load_dotenv
import json
import matplotlib.pyplot as plt
from openai import OpenAI
import os
import pandas as pd
from pydantic import BaseModel
import textwrap

import config

load_dotenv()
client = OpenAI()

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
df = pd.read_csv('processed/2023.csv')
df = df[~df['Empty Response']].replace("-", pd.NA)

In [ ]:
df.head()

In [ ]:
weight_by_parents = False


def calculate_question_totals(df_):
    results = []
    filters = {
        "Year 1 Families": pd.to_numeric(df_["Years at GVCA"]) == 1,
        "Not Year 1 Families": pd.to_numeric(df_["Years at GVCA"]) > 1,
        "Year 3 or Less Families": pd.to_numeric(df_["Years at GVCA"]) <= 3,
        "Year 4 or More Families": pd.to_numeric(df_["Years at GVCA"]) > 3,
        "Minority": df_["Minority"] == "Yes",
        "Not Minority": df_["Minority"] != "Yes",
        "Support": df_["IEP, 504, ALP, or Read"] == "Yes",
        "Not Support": df_["IEP, 504, ALP, or Read"] != "Yes",
    }

    for question in config.questions_for_each_school_level:
        response_levels = config.question_responses.get(question, [])
        
        for response in response_levels:
            response_data = {"Question": question, "Response": response}
            
            schoolwide_counts, schoolwide_total = _calculate_totals(df_, question, response, config.levels, weight_by_parents)
            response_data.update(_format_counts_and_percentages("total", schoolwide_counts, schoolwide_total, response))
            
            for level in config.levels:
                level_counts, level_total = _calculate_totals(df_, question, response, [level], weight_by_parents)
                response_data.update(_format_counts_and_percentages(level, level_counts, level_total, response))
            
            for filter_name, filter_condition in filters.items():
                filtered_counts, filtered_total = _calculate_totals(df_[filter_condition], question, response, config.levels, weight_by_parents)
                response_data.update(_format_counts_and_percentages(filter_name, filtered_counts, filtered_total, response))
            
            results.append(response_data)
    
    return pd.DataFrame(results)

def _calculate_totals(df_, question, response, levels, weight_by_parents):
    """Helper to calculate counts and totals for given levels."""
    totals = {}
    overall_total = 0

    for level in levels:
        column_name = f"({level}) {question}"
        if column_name in df_.columns:
            filtered_df = df_[df_[column_name] == response]

            if weight_by_parents:
                response_sum = filtered_df["N Parents Represented"].astype(float).sum()
                level_total = df_[~df_[column_name].isna()]["N Parents Represented"].astype(float).sum()
            else:
                response_sum = len(filtered_df)
                level_total = len(df_[column_name].dropna())

            totals[response] = totals.get(response, 0) + response_sum
            overall_total += level_total

    return totals, overall_total

def _format_counts_and_percentages(label, counts, total, response):
    """Helper to format counts and percentages for a given response."""
    count = counts.get(response, 0)
    percentage = (count / total) * 100 if total > 0 else 0
    return {f"N_{label}": count, f"%_{label}": percentage}

rolled_up_data = calculate_question_totals(df)
rolled_up_data.to_excel("2023_rolled_up_data.xlsx", index=False)
rolled_up_data

In [ ]:
def calculate_top_two_from_rollup(rolled_up_data):
    results = []

    for question in config.questions_for_each_school_level:
        top_two_responses = config.question_responses.get(question, [])[:2]  # Get first two satisfaction levels
        
        # Filter the rolled-up data for relevant responses
        filtered_data = rolled_up_data[(rolled_up_data["Question"] == question)]
            # ()

        if filtered_data.empty:
            continue

        response_data = {"Question": question}

        # Aggregate across all relevant columns (e.g., total, school levels, and filters)
        for column in rolled_up_data.columns:
            if column.startswith("N_"):  # Sum counts for relevant responses
                total_count = filtered_data[column].sum()
                total_responses = filtered_data[filtered_data["Response"].isin(top_two_responses)][column].sum()

                response_data[column] = total_responses
                response_data[column.replace("N_", "%_")] = (total_responses / total_count) * 100 if total_responses > 0 else 0

        results.append(response_data)

    return pd.DataFrame(results)

top_two = calculate_top_two_rollup(rolled_up_data)
top_two


In [ ]:
def create_stacked_bar_chart(
    title: str,
    x_axis_label: str,
    x_data_labels: list,
    proportions: dict,
    savefig=False,
    subfolder="artifacts",
    
) -> None:
    """
    Save a stacked bar chart to ./artifacts/
    """
    r1 = [proportions[question][3] for question in config.questions_for_each_school_level if question not in config.has_free_response]
    r2 = [proportions[question][2] for question in config.questions_for_each_school_level if question not in config.has_free_response]
    r3 = [proportions[question][1] for question in config.questions_for_each_school_level if question not in config.has_free_response]
    r4 = [proportions[question][0] for question in config.questions_for_each_school_level if question not in config.has_free_response]

    fig, ax = plt.subplots(1, figsize = (20, 8))
    ax.bar(
        x_data_labels,
        r4,
        label="Very",
        color="#6caf40",
        bottom=[q1 + q2 + q3 for q1, q2, q3 in zip(r1, r2, r3)],
    )
    ax.bar(
        x_data_labels,
        r3,
        label="Satisfied",
        color="#4080af",
        bottom=[q1 + q2 for q1, q2 in zip(r1, r2)],
    )
    ax.bar(x_data_labels, r2, label="Somewhat", color="#f6c100", bottom=r1)
    ax.bar(x_data_labels, r1, label="Not", color="#ae3f3f")

    ax.set_title(title)
    ax.set_xlabel(x_axis_label)
    ax.set_ylabel("Proportion")

    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    
    # Put a legend to the right of the current axis
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.tight_layout()

    if savefig:
        if not os.path.exists(subfolder):
            os.mkdir(subfolder)
        plt.savefig(
            f"{subfolder}/{title}",
            transparent=True,
        )
    plt.show()

def to_proportions_and_labels(df, col):
    print(col)
    response_proportions = (
        df.groupby(["Question", "Response"])[col]
        .sum()
        .unstack(fill_value=0)  # Pivot so that each response is a column
    )

    # Normalize by row sum to get proportions
    response_proportions = response_proportions.div(response_proportions.sum(axis=1), axis=0)

    proportions = {}
    labels = []
    for question in config.questions_for_each_school_level:
        score = 0
        if question in config.has_free_response:
            continue
        proportions[question] = []
        n_options = len(config.question_responses.get(question, []))
        for i, response in enumerate(config.question_responses.get(question, [])):
            proportion = response_proportions.loc[question, response]
            proportions[question].append(proportion)
            score += proportion*(n_options-i)
        labels.append(f"{textwrap.fill(question, 35)}\n({score:.2f})")

    return proportions, labels

def plot_sequence(grouping, df_, savefig=False):
    splits = [
        ("All Responses", "N_total"),
        ("Grammar Responses", "N_Grammar"),
        ("Middle Responses", "N_Middle"),
        ("Upper Responses", "N_Upper"),
        ("Minority Responses", "N_Minority"),
        ("Support Responses", "N_Support"),
    ]

    for split in splits:
        proportions, labels = to_proportions_and_labels(df_, split[1])
        create_stacked_bar_chart(
            f"{grouping} {split[0]}",
            "Response Summary",
            labels,
            proportions,
            savefig=savefig,
        )

In [ ]:
plot_sequence("Total", rolled_up_data)

In [ ]:
newer_families_rolled_up_data = calculate_question_totals(df[pd.to_numeric(df["Years at GVCA"]) <= 3])
plot_sequence("Newer Families", newer_families_rolled_up_data)

In [ ]:
older_families_rolled_up_data = calculate_question_totals(df[pd.to_numeric(df["Years at GVCA"]) > 3])
plot_sequence("Older Families", older_families_rolled_up_data)

In [ ]:
df["(Generic) Please provide us with examples of how GVCA can better serve you and your family."][12]

In [ ]:
"""
4o-mini allows you to process the entire data set, but is a lower quality model
4o can handle subsets of the whole data, but theoretically produces better results

evaluating the outcome indicates a superior response from 4o-mini
though using 4o to enhance the taxonomies given both the 4o (detailed) and the 4o-mini (global) produces a taxonomy that is far superior
"""

class SurveyTaxonomyTag(BaseModel):
    tag: str
    explanation: str

class SurveyTaxonomy(BaseModel):
    tags: list[SurveyTaxonomyTag]

taxonomy_prompt = f"""
You are processing parent survey data from a school accountability committee. The parents were asked to provide feedback about which things have been working well and which things could be improved.

Before diving into the data set you want to generate a set of tags that could be used to help contextualize and filter this free response data. It is acceptable if multiple tags could be applied to the same input. 
But you should attempt to broadly cover the survey results with this tag taxonomy.

Each tag should be relevant to the content of the record and should consist of no more than 4 words. 

In addition to each tag, please provide a brief explanation of the tag (no more than 30 words). Generate no more than 30 tags in total.

Here are the records:
{bulleted_free_responses}

Please generate the tags and explanations.
"""

text = df["Total Free Response"].dropna().tolist()
bulleted_free_responses = "\n".join(["- "+re.sub(r'\s+', ' ', t.replace("\n", " ")).strip() for t in text[:50]])

completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "developer", "content": "You are a helpful assistant."},
        {"role": "user", "content": taxonomy_prompt}
    ],
    response_format=SurveyTaxonomy,
)

for tag in completion.choices[0].message.parsed.tags:
    print(f"{tag.tag}: {tag.explanation}")

In [ ]:
prompt2 = f"""
You are processing parent survey data from a school accountability committee. The parents were asked to provide feedback about which things have been working well and which things could be improved.

You have a taxonomy of classifications you want to label free response data with which is:
{completion.choices[0].message.content}

You want to evaluate the following and identify which categories should be applied to this free response input
```input
{df["(Generic) Please provide us with examples of how GVCA can better serve you and your family."][12]}
```

it is important to only provide the classifications that actually match to this free response and to not deviate from the provided taxonomy.
please produce as a simple list
"""
prompt2

In [ ]:
completion2 = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "developer", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt2}
  ],
    n=10,
)

In [ ]:
completion2

In [ ]:
len(completion2.choices)

In [ ]:
for i in range(10):
    print(completion2.choices[i].message.content)

In [ ]:
df["(Grammar) What makes GVCA a good choice for you and your family?"][0]